# Source: NCAA

In [1]:
#hide
import core_constants as cc
import functions as fx
import json

## Set Notebook Settings

In [2]:
#Load NCAA Site Schema
schoolsList = json.loads(open('..//config//schools.json', "r").read())
ncaaDates = json.loads(open('..//config//ncaa_dates.json', "r").read())

## Get & Save the Teams Page HTML
#### Source: https://stats.ncaa.org/team/392/roster/15280
> This page contains a roster for any given team in any year (using their own ids)

In [3]:
fx.get_NCAA(schoolsList, ncaaDates)

## Process Local NCAA HTML Files

> All of this processing is done locally, using the files saved in the previous few steps.  This creates an exhaustive store of all the fields grabbed from the scrapes.

In [4]:
conferences = cc.get_availableConferences()

cc.save_records('scrapedData', 'ncaa', fx.process_NCAA(conferences))

## Summarize the Dataset

> We don't need repetitive fields across the various datasets (ie - I don't need height coming back from 3 sources).  This step strips to only what I care about for the master print out.

In [5]:
outputDir = '..//summarizedData//'
dataset = 'ncaa'

with open(outputDir + dataset + ".json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.summarize_NCAA()))